# 网络安全基线
* 安全基线是信息安全行业的一项标准工作，属于安全服务范畴
* 安全基线旨在为企业建立标准或独立的基本安全配置原则
* 以安全原则来检查服务器、网络、应用程序、中间件、数据库等IT基础设施，从而保障所有的设施都具备了最基本的安全条件
* 网络安全基线是指针对网络设备的安全基线扫描工作，确保网络设备的各个配置项都进行了安全的设定

In [1]:
import re
import helper.ios_security_line as secline
from netmiko import ConnectHandler
from pyecharts.charts import Liquid

## 设定运维信息
* 作业：建立自己的项目时改为从数据库读取信息

In [2]:
device_info = {
    "device_type": "cisco_ios",
    "ip" : "192.168.1.60", 
    "port" : "22", 
    "username" : "ciscouser",
    "password" : "cisco@123",
    "secret" : "cisco123",
}

## 建立安全基线检查类
* 这里使用了IOS基线脚本
* 希望各位能在今后的工作中持续维护基线脚本
* 每家企业都需要有个性化的安全标准，安全服务于业务

In [7]:
class ios_sec_checker():
    def __init__(self, device_info):
        self.server = ConnectHandler(**device_info)
        self.server.enable()
        print("设备连接初始化完成")
        self.config = self._get_config()
        self.sec_score = 100
    
    def _get_config(self):
        return self.server.send_command("show run")
        
    def _close_session(self):
        self.server.disconnect()
        
    def admin_login_sec_check(self):
        if not secline.enable_secret_check(self.config):
            print("  未检测到enable secret")
            self.sec_score -= 10
        if not secline.user_secret_check(self.config):
            print("  未检测到user secret")
            self.sec_score -= 10
        if not secline.vty_ssh_check(self.config):
            print("  VTY线路未开启SSH")
            self.sec_score -= 10
        if not secline.vty_acl_check(self.config):
            print("  VTY未限制接入源")
            self.sec_score -= 10
                
    def ospf_sec_check(self):
        if not secline.ospf_sec_check(self.config):
            print("  OSPF未启用协议加密")
            self.sec_score -= 10
            
    def check_useless_service(self):
        if not secline.cdp_disable_check(self.config):
            print("  CDP服务未关闭")
            self.sec_score -= 10
        if not secline.pad_disable_check(self.config):
            print("  PAD未关闭")
            self.sec_score -= 10
            
    def display_score(self):
        show_data = self.sec_score * 0.01
        liq_sight = Liquid()
        liq_sight.add("NET-DEVICE-SEC", [show_data, show_data], is_outline_show=False)
        return liq_sight
            
    def run_all(self):
        print("-"*15)
        self.admin_login_sec_check()
        self.ospf_sec_check()
        self.check_useless_service()
        print("-"*15)
        if self.sec_score == 100:
            print("恭喜！网络设备完全符合安全基线")
        elif 70 <= self.sec_score < 100:
            print("健康状况良好，请酌情修复问题")
        else:
            print("健康状况较差，请及时修复问题")
        self._close_session()
        return self.display_score()

## 执行安全基线检查

In [15]:
sec_checker = ios_sec_checker(device_info)
display = sec_checker.run_all()
display.render_notebook()

设备连接初始化完成
---------------
  OSPF未启用协议加密
  CDP服务未关闭
  PAD未关闭
---------------
健康状况良好，请酌情修复问题
